<a href="https://colab.research.google.com/github/aidanjmaldonado/penny-stock-lstm/blob/main/penny_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [381]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
import sqlite3
import requests
import sys
from library.DataSetProcessor import DataSetProcessor
import matplotlib.pyplot as plt

In [382]:
# Constants
SEQUENCE_LENGTH = 78 # 1 day long
PREDICTION_LENGTH = 78 # 1 day long
NUM_FEATURES = 2 # close, volume

# Create database

In [383]:
# Download database from github repository
historical_url = "https://raw.githubusercontent.com/CSE-115-UCSC/penny-stock-lstm/main/historicaldata.db"
scrape_request = requests.get(historical_url)

try:
  # Contingent on request status
  scrape_request = requests.get(historical_url)
  scrape_request.raise_for_status()

  # Create local database from pull, name 'historicaldata.db'
  with open("historical.db", "wb") as db_file:
    db_file.write(scrape_request.content)
    
  print("Request to download database succeeded")


except:
  # Report failed request status
  sys.stderr.write("Request to download database failed")

Request to download database succeeded


In [384]:
# Connect to SQlite database
try:
    db = 'historical.db'
    sqliteConnection = sqlite3.connect(db)
    cursor = sqliteConnection.cursor()
    print(f'SQlite connected with {db}')

except:
    # Report failed request status
    sys.stderr.write("Failed to connect to database")

SQlite connected with historical.db


In [385]:
# Query all historical stock data from Database
try:
    query = f"SELECT * FROM all_historical;"
    cursor.execute(query)
    if cursor.fetchone() is None:
        raise Exception("No results")

    print(f"Success querying all historical")
    # Turn SQlite Database into Pandas Dataframe
    data = pd.read_sql_query(query, sqliteConnection)

except:
    sys.stderr.write(f"Failed to select all historical")

Success querying all historical


In [386]:
# Using the column 'time' (millisecond) add a new column 'dates' with datetime
dates = pd.to_datetime(data['time'], unit='ms')
tickers = data['ticker']
dates = dates.dt.tz_localize('UTC').dt.tz_convert('US/Pacific')
dates = dates.dt.tz_localize(None)

# Dataset Normalization

In [387]:
normalized_data = pd.DataFrame(columns=['close','volume'])

In [388]:
data_by_ticker = {}
for ticker in data['ticker'].unique():
    data_by_ticker[ticker] = data[data['ticker'] == ticker].copy()
    data_by_ticker[ticker]['close_norm'] = data_by_ticker[ticker]['close'] / data_by_ticker[ticker]['close'].max() #Normalized closing price data

In [389]:
for key in data_by_ticker:
    # Create a temporary DataFrame to hold the current data
    temp_df = pd.DataFrame({
        'close': data_by_ticker[key]['close_norm'],
        'volume': data_by_ticker[key]['volume']
    })
    
    # Concatenate the temporary DataFrame to the normalized_data DataFrame
    normalized_data = pd.concat([normalized_data, temp_df], ignore_index=True)

# Optionally, you can reset the index if needed
normalized_data.reset_index(drop=True, inplace=True)

/var/folders/s4/t5945xms4x52fkpsd0m4v9340000gn/T/ipykernel_446/1044765597.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  normalized_data = pd.concat([normalized_data, temp_df], ignore_index=True)


# Train on all historical stock data, sequenced

In [390]:

"""Generate arrays filled with one-day-long sequences from the normalized dataset

Arguments:
    - data: stock dataset with 2 columns:
        - close prices normalized (0, 1) 
        - volume
    - dates: array of every milisecond timestamp converted to dateTime objects
    - tickers: array of every timestamp's corresponding ticker

Returns:
    - x: array of one-day-long sequences of the normalized dataset for training
    - y: array of one-day-long seuqences of the immediate day after for predicting

"""
def create_sequences(data, dates, tickers):
    
    # stores sequences to be returned
    xs, ys = [], []
    # index refers to the start of a day, therefore start of a 'sequence'
    index = 0 
     # keeps track of the number of valid sequences for debugging purposes
    count = 0

    # Loop until the end of database, stopping 2 days in advance to make room for last 'context' day and it's corresponding 'prediction' day
    while index < len(data) - SEQUENCE_LENGTH - PREDICTION_LENGTH + 1:

        # Check if sequence is within a single day (day start == day end) and (ticker start == ticker end)
        if dates[index].date() == dates[index + SEQUENCE_LENGTH].date() and tickers[index] == tickers[index + SEQUENCE_LENGTH]:

            # append current day (index -> index+SEQ) to x, and append next day (index + SEQ -> index + SEQ + PRE) to y
            xs.append(data.iloc[index:index + SEQUENCE_LENGTH])  # Use past data for features
            ys.append(data.iloc[index + SEQUENCE_LENGTH:index + SEQUENCE_LENGTH + PREDICTION_LENGTH, 0])  # Only predict 'close' prices

            # move index to start of the next day
            index += SEQUENCE_LENGTH
            count += 1
        
        # Move index to the start of the next 
        else: # Note: This is the discarding section, can be modified to be "imputed" via extending the last known close value until end of day.

            # move newindex to the start of the next day
            newindex = index
            while dates[newindex].date() == dates[newindex + 1].date():
                newindex += 1
            newindex += 1
            
            # once newindex reaches next morning, set index to match
            index = newindex

    # print the number of valid days found for debugging purposes, return arrays of sequences            
    print("Valid days:", count)
    return np.array(xs), np.array(ys)

# Create sequences from normalized data
x, y = create_sequences(normalized_data, dates, tickers) #Creating the input and grouth truth data from create_sequences function

Valid days: 294


# Training Pipeline

In [391]:

# Split data into 80% / 20% training and testing groups
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Type adjustment string -> float
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)

# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(SEQUENCE_LENGTH, NUM_FEATURES)))
# Need both layers because return_sequences will send its output to another LSTM layer which is required before sending to Dense layer 
model.add(LSTM(50))
 # Makes readable by NN, NN doesn't predict on sequences so it needs single dimension values
model.add(Dense(40, activation='relu'))
# Prevents overfitting
model.add(Dropout(0.1))
# Takes the results from the last LSTM layer and predicts the stock prices for PREDICTION_LENGTH steps ahead
model.add(Dense(PREDICTION_LENGTH)) 
#Compiles the model with an adam optimizer and a mean squared error loss function
model.compile(optimizer='adam', loss='mse') 

# Train the model with early stopping to prevent over fitting
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100


/Users/aidanmaldonado/Documents/PennyStock/Website/stock-prediction/venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.2224 - val_loss: 0.0374
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1513 - val_loss: 0.0434
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1271 - val_loss: 0.0456
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1051 - val_loss: 0.0409
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0926 - val_loss: 0.0358
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0749 - val_loss: 0.0428
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0754 - val_loss: 0.0444
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0636 - val_loss: 0.0385
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0567 - val_loss: 0.0348
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0656 - val_loss: 0.0552
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0611 - val_loss: 0.0287
Epoch 12/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0681 - val_loss: 0.0689
E

In [393]:
# Save the model weights to an external file
model.save('model.h5')